## 准备数据

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [1]:
# class myModel:
#     def __init__(self):
#         ####################
#         '''声明模型对应的参数'''
#         ####################
#     def __call__(self, x):
#         ####################
#         '''实现模型函数体，返回未归一化的logits'''
#         ####################
#         return logits
        


class myModel:
    def __init__(self):
        # 声明模型对应的参数
        self.W1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.truncated_normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])  # Flatten the input
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # First layer
        logits = tf.matmul(h1, self.W2) + self.b2  # Output layer
        return logits


model = myModel()

optimizer = optimizers.Adam()

NameError: name 'tf' is not defined

## 计算 loss

In [12]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 561.0778 ; accuracy 0.00023333334
epoch 1 : loss 522.6978 ; accuracy 0.00091666664
epoch 2 : loss 489.7846 ; accuracy 0.0025833333
epoch 3 : loss 460.9614 ; accuracy 0.0057333335
epoch 4 : loss 435.13544 ; accuracy 0.011133334
epoch 5 : loss 410.89774 ; accuracy 0.016533334
epoch 6 : loss 387.11734 ; accuracy 0.022933334
epoch 7 : loss 363.63397 ; accuracy 0.030516667


: 